In [ ]:
%matplotlib inline

from consts import *
import os,cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Polygon, Rectangle
from PIL import Image
from keras.models import Model
from joblib import dump, load

dest_dir = base_path + 'images/rcnn_out/'


In [ ]:
# load up the CNN weights
weights_dir = base_path + 'weights/'

from CNN_model import model

# model.load_weights(weights_dir + 'cnn_' + str(CLASS_ID) + '.h5')
model.load_weights(weights_dir + 'cnn_0_120_good.h5')
intermed_model = Model(inputs=model.input,
                   outputs=model.layers[-2].output)
intermed_model.summary()


In [ ]:
# loading up the SVM
svm_weights_path = base_path + 'weights/svm_0_1024.joblib'
clf = load(svm_weights_path) 

In [ ]:
USE_SVM = False

from skimage import io

if not os.path.exists(dest_dir):
    os.mkdir(dest_dir)

ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

n = 0
for j in range(16,29):
    img_path = base_path + 'images/rcnn_test/test' + str(j) + '.jpg'
    # run selective search
    image = cv2.imread(img_path)[...,::-1] # switch to rgb

    # initialise selective search

    ss.setBaseImage(image)
    ss.switchToSelectiveSearchFast()
    ssresults = ss.process()
    print(len(ssresults))

    if USE_GREY:
        imout = io.imread(img_path, as_gray=True)
        imout = np.expand_dims(imout, axis=2)
    else:
        imout = image
    
    plt.figure(figsize=(18,18))
    ax = plt.gca()
    ax.imshow(image)

    
    crop_list = []
    
    for i,result in enumerate(ssresults):
        if i >= 2000:
            break
        x,y,w,h = result

        if w < MIN_SIZE or h < MIN_SIZE:
            continue
        rect = Rectangle([x ,y], w, h, fill=False, 
                                color='green', lw=0.5)
        ax.add_patch(rect)

        crop = imout[y:y+h,x:x+w]
        resized = cv2.resize(crop, (IMG_SIZE,IMG_SIZE), interpolation = cv2.INTER_AREA)
        if not USE_GREY:
            resized_norm = (resized/255) - 0.5
        else:
            resized_norm = resized - 0.5
            
        resized_norm = np.expand_dims(resized_norm, axis=0)
        # print(resized_norm[0][0:10])

        layer_output = np.array(intermed_model(resized_norm))
        # print(layer_output[0][0:20])

        svm_prob = clf.predict_proba(layer_output)


        if svm_prob[0][1] > 0.2:
            crop = [x, y, w, h]
            crop_list.append([crop, svm_prob[0][1], resized])
    
#     sorted_crop_list = list(sorted(crop_list, key=lambda item:item[1], reverse=True))

#     max_k = len(sorted_crop_list)
#     if max_k > 3:
#         max_k = 3
        
#     for k, crop_img in enumerate(sorted_crop_list):
#         if k >= max_k:
#             break
#         crop, score, resized = crop_img
#         print(score)
#         x, y, w, h = crop
#         if USE_GREY:
#             resized = np.uint8(resized * 255)
#             pil_img = Image.fromarray(resized, 'L')
#         else:
#             pil_img = Image.fromarray(resized)
#         pil_img.save(dest_dir + 'negative_' + str(n) +'.png')
#         n += 1
        
#         rect = Rectangle([x ,y], w, h, fill=False, 
#                             color='red', lw=1) 
#         ax.add_patch(rect)
    save_file = dest_dir + 'test_' + str(j) + '.png'
    print('\n')
    plt.savefig(save_file, bbox_inches='tight')
